### The following code demonstrates the calculation of disparity map and point cloud

#### Import rquired packages

In [1]:
import numpy as np
import cv2
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#### Read left and right images

In [2]:
img_left = cv2.imread("left.png", 0)
img_right = cv2.imread("right.png", 0)

#### Assign camera intrinsics and baseline parameters

In [3]:
fx, fy, cx, cy = 718.856, 718.856, 607.1928, 185.2157
b = 0.573

#### SGBM is senstive to parameters, please tune according to requirements.


In [4]:
window_size = 9
min_disp = 0
num_disp = 96
stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=window_size,
    P1=8 * 3 * window_size ** 2,
    P2=32 * 3 * window_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=63,
    speckleWindowSize=10,
    speckleRange=100,
    preFilterCap=32,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

In [5]:
disparity_sgbm = stereo.compute(img_left, img_right).astype(np.float32) / 16.0
disparity_sgbm = (disparity_sgbm - min_disp) / num_disp

#### Compute the point cloud

In [6]:
points = []
colors = []
for v in range(img_left.shape[0]):
    for u in range(img_left.shape[1]):
        if disparity_sgbm[v, u] <= 0.0 or disparity_sgbm[v, u] >= 96.0:
            continue

        point = np.zeros(3)
        colors.append(img_left[v, u] / 255.0)
        
        x = (u - cx) / fx
        y = (v - cy) / fy
        depth = fx * b / (disparity_sgbm[v, u])
        point[0] = x * depth
        point[1] = y * depth
        point[2] = depth

        points.append(point)

#### Display Right image and Left image with Disparity

In [7]:
cv2.imshow("Left", img_left)
cv2.imshow("Right", img_right)
cv2.imshow("Disparity", disparity_sgbm)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Assign and visualise point cloud using open3d

In [8]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(np.stack([colors, colors, colors], axis=1))

o3d.visualization.draw_geometries([pcd])